In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd /mnt/data/anhvth5/projects/eedi

/mnt/data/anhvth5/projects/eedi


In [3]:
import os
import dspy

In [4]:
from speedy_utils.all import *
import random
from dotenv import load_dotenv
import dspy
from llm_utils import *
from eedi import dspy as edspy
from eedi.common import *
from eedi.dspy.generate_train_data import SignatureProblemGenerator
import argparse

from eedi.models.openai_embed import get_miscon_db_openai

load_dotenv()
setup_logger("E")


2024-12-04 14:33:58.181 | INFO     | eedi.common:<module>:429 - common: ['VectorDBRetriever', 'prepare_db_train_val', 'preproc_df', 'get_df_parsed', 'format_df_generic', 'compute_metric', 'mine_hard_negatives', 'mine_hard_negatives', 'extract_tag', 'format_input_v2', 'format_input_v3', 'TEMPLATE_INPUT_V3', 'construct_base_prompt_v3', 'get_synthetic_df', 'format_docs', 'get_parsed_df_v3', 'df_mapping', 'get_parsed_df_v4', 'get_openai_misconception_db', 'update_train_df', 'load_latest_df', 'OpenAIModel', 'LocalLM']


In [5]:
edspy.LocalLM.scan_model_ports()

{'QW72B': [2802]}

In [6]:
# lm_ds = dspy.LM(base_url="https://api.deepseek.com", model="openai/deepseek-chat", api_key='sk-0831d94757284b57bb7f31f9173b2a47')
# lm_ds('who are you')

In [7]:
lm = edspy.LocalLM("QW72B")
dspy.settings.configure(lm=lm)
db = get_miscon_db_openai()


In [8]:
from dataclasses import dataclass, field
import argparse

@dataclass
class Config:
    seed: int = 42
    topk: int = 30
    max_examples: int = 5
    skip_exact_match: bool = field(default=False)
args = Config()

In [9]:
# edspy.LocalLM.scan_model_ports()

In [10]:
from eedi.common import df_mapping
df_eval = pd.read_parquet('./outputs/llm_hn_qw7b/10_59_more_synthetic_fewshot/awq/eval_results_embed_train.parquet')
df_eval.head(3)

,Query,Prompt,MisconceptionId,Recall,MAP@25,RetrievedIDs,RetrievedScores
QuestionId_Answer,,,,,,,
0_B,<instruct>Given a math multiple-choice problem...,Question: \[\n3 \times 2+4-5\n\]\nWhere do the...,NaN,NaN,NaN,"[1507, 1345, 2306, 1005, 2532, 315, 706, 1672,...","[0.39689746499061584, 0.3422924876213074, 0.29..."
0_C,<instruct>Given a math multiple-choice problem...,Question: \[\n3 \times 2+4-5\n\]\nWhere do the...,NaN,NaN,NaN,"[1999, 1345, 987, 1507, 2532, 1005, 2306, 2488...","[0.3276568651199341, 0.3195399045944214, 0.318..."
0_D,<instruct>Given a math multiple-choice problem...,Question: \[\n3 \times 2+4-5\n\]\nWhere do the...,1672.0,1.0,0.5,"[2532, 1672, 1005, 1345, 328, 1507, 987, 1516,...","[0.3396575450897217, 0.33421608805656433, 0.30..."


In [11]:
class SignatureProblemGenerator(dspy.Signature):
    """
    Generate a plausible math problem based on a given misconception.
    NOTE: Be *creative* when generate the questions. Just make sure it align with the *given misconception*.
    Create 3 questions with different difficulty levels
    """

    MISCONCEPTION_NAME = dspy.InputField(
        desc="A specific misconception related to a mathematical concept"
    )
    PLANING = dspy.OutputField(
        desc="This is where you create a plan for generating question/correct/incorrect answer before actually generating it. Focus on the input field to generate a relevant plan, evaluate your plan and make changes all in here"
    )
    QUESTION = dspy.OutputField(
        desc="A generated math problem consist of Question, SubjectName, and ConstructName"
    )
    CORRECT_ANSWER = dspy.OutputField(
        desc="The correct answer to the generated math problem."
    )
    STUDENT_WRONG_ANSWER = dspy.OutputField(
        desc="An incorrect answer that a student might give based on the misconception."
    )


In [12]:
# plan_reason_df

In [13]:
def get_demos(miscon, args, plan_reason_df):
    examples = []

    for similar_misconception, score in db(miscon):
        if args.skip_exact_match and similar_misconception == miscon:
            continue
        ids = plan_reason_df["MISCONCEPTION_NAME"] == similar_misconception
        if ids.sum() > 0:
            examples.append(plan_reason_df[ids].sample(1, random_state=args.seed).iloc[0].to_dict())
    return {"miscon": miscon, "demos": examples}


def generate_problem(input):
    try:
        pred = prog(MISCONCEPTION_NAME=input["miscon"], demos=input["demos"])
        pred = pred.toDict()
        pred["MISCONCEPTION_NAME"] = input["miscon"]
        return pred
    except Exception as e:
        return {"e": str(e)}


os.makedirs("./data/synthetic/", exist_ok=1)

prog = dspy.Predict(SignatureProblemGenerator)

args.seed = args.seed
args.topk = args.topk
out_path = f"./data/synthetic/seed_{args.seed}.parquet"

df_2610 = pd.read_parquet("./data/train_1026.parquet")
df_mapping = pd.read_csv("./data/misconception_mapping.csv")
MISCONCEPTION_NAMES = df_mapping["MisconceptionName"].tolist()
df_parsed = pd.read_parquet("./data/df_parsed_1026_22h_with_rerank_result.parquet")
plan_reason_df = pd.read_parquet("./data/plan_reason_df.parquet")


def generate_problem(input):
    pred = prog(MISCONCEPTION_NAME=input["miscon"], demos=input["demos"])
    pred = pred.toDict()
    pred["MISCONCEPTION_NAME"] = input["miscon"]
    return pred



response_with_demos = get_demos(db.texts[0], args, plan_reason_df)
response1 = generate_problem(response_with_demos)

In [14]:
# lm72.ih()

In [15]:
# df_eval

In [16]:
miscon_id_with_label = df_eval['MisconceptionId'].dropna().apply(int).unique()
misconceptions = df_mapping['MisconceptionName'].tolist()
miscon_id_without_label = list(set(df_mapping['MisconceptionId'].apply(int).unique())- set(miscon_id_with_label))

In [17]:
len(miscon_id_without_label)

983

### BUILD LONG DEMOS 

In [18]:
df_eval.head(3)

,Query,Prompt,MisconceptionId,Recall,MAP@25,RetrievedIDs,RetrievedScores
QuestionId_Answer,,,,,,,
0_B,<instruct>Given a math multiple-choice problem...,Question: \[\n3 \times 2+4-5\n\]\nWhere do the...,NaN,NaN,NaN,"[1507, 1345, 2306, 1005, 2532, 315, 706, 1672,...","[0.39689746499061584, 0.3422924876213074, 0.29..."
0_C,<instruct>Given a math multiple-choice problem...,Question: \[\n3 \times 2+4-5\n\]\nWhere do the...,NaN,NaN,NaN,"[1999, 1345, 987, 1507, 2532, 1005, 2306, 2488...","[0.3276568651199341, 0.3195399045944214, 0.318..."
0_D,<instruct>Given a math multiple-choice problem...,Question: \[\n3 \times 2+4-5\n\]\nWhere do the...,1672.0,1.0,0.5,"[2532, 1672, 1005, 1345, 328, 1507, 987, 1516,...","[0.3396575450897217, 0.33421608805656433, 0.30..."


In [19]:
import torch
miscon_id_example_score = np.zeros([len(df_mapping), len(df_eval)])
rets = np.array(df_eval['RetrievedIDs'].values.tolist())

for example_id, misconception_ids in enumerate(rets):
    for rank, miscon_id in enumerate(misconception_ids):
        score = 3/(3+rank)
        miscon_id_example_score[miscon_id, example_id] += score
        
miscon_id_example_score = torch.tensor(miscon_id_example_score)

In [20]:
plan_reason_df.dropna()

,QUESTION,CORRECT_ANSWER,STUDENT_WRONG_ANSWER,MISCONCEPTION_NAME,PROMPT,reasoning,PLANING
qid_choice,,,,,,,
0_D,Question: \[\n3 \times 2+4-5\n\]\nWhere do the...,\( 3 \times(2+4)-5 \),Does not need brackets,"Confuses the order of operations, believes add...",Question: \[\n3 \times 2+4-5\n\]\nWhere do the...,Students with this misconception may incorrect...,1. Identify the misconception where students b...
1_A,"Question: Simplify the following, if possible:...",Does not simplify,\( m+1 \),Does not know that to factorise a quadratic ex...,"Question: Simplify the following, if possible:...",Students with this misconception may not under...,1. Identify the misconception where students d...
1_B,"Question: Simplify the following, if possible:...",Does not simplify,\( m+2 \),Thinks that when you cancel identical terms fr...,"Question: Simplify the following, if possible:...",Students with this misconception may incorrect...,1. Identify the misconception where students c...
1_C,"Question: Simplify the following, if possible:...",Does not simplify,\( m-1 \),Does not know that to factorise a quadratic ex...,"Question: Simplify the following, if possible:...",Students with this misconception may not under...,1. Identify the misconception where students d...
2_A,Question: Tom and Katie are discussing the \( ...,Only\nKatie,Only\nTom,Believes if you changed all values by the same...,Question: Tom and Katie are discussing the \( ...,Students with this misconception may incorrect...,1. Identify the misconception where students t...
...,...,...,...,...,...,...,...
1867_C,Question: Tom and Katie are discussing congrue...,Only Katie,Both Tom and Katie,Thinks congruent shapes are the same shape but...,Question: Tom and Katie are discussing congrue...,Students with this misconception may incorrect...,1. Identify the misconception where students c...
1867_D,Question: Tom and Katie are discussing congrue...,Only Katie,Neither is correct,Thinks congruent shapes are the same shape but...,Question: Tom and Katie are discussing congrue...,Students with this misconception may incorrect...,1. Identify the misconception where students c...
1868_A,Question: Jo and Paul are arguing about how to...,Only Paul,Only\nJo,Forgets to give the centre of rotation,Question: Jo and Paul are arguing about how to...,Students with this misconception may forget to...,1. Identify the misconception where students f...


In [21]:
from collections import defaultdict
import numpy as np
import random

def generate(i):
    miscon_id, round_seed = i
    # Ensure the miscon_id is a hashable type (convert to string if needed)
    seed_value = str(miscon_id)+f'_{round_seed}'
    
    # Initialize a seeded random generator based on miscon_id
    random_generator = random.Random(seed_value)
    np_random_state = np.random.RandomState(random_generator.randint(0, 2**32 - 1))
    
    # Step 1: Sort scores and get top indices
    scores, indices = miscon_id_example_score[miscon_id].sort(0, True)
    top_indices = df_eval.iloc[indices[:100]].index.tolist()
    
    # Step 2: Filter indices to those present in plan_reason_df
    valid_indices = [idx for idx in top_indices if idx in plan_reason_df.index]
    
    # Step 3: Group by question ID
    qid_to_indices = defaultdict(list)
    for index in valid_indices:
        question_id, answer = index.split('_')
        qid_to_indices[question_id].append(index)
    
    # Step 4: Select a single index per question ID
    selected_indices = [random_generator.choice(indices) for indices in qid_to_indices.values()]
    
    # Step 5: Filter and prepare demonstration records
    demos_df = plan_reason_df.loc[selected_indices].dropna(subset=['MISCONCEPTION_NAME'])
    random_sample_ids = np_random_state.choice(len(demos_df), 5, replace=False)
    demos = demos_df.iloc[random_sample_ids].to_dict('records')
    
    # Step 6: Generate input and problem
    misconception = misconceptions[miscon_id]
    input_data = {'miscon': misconception, 'demos': demos}
    response = generate_problem(input_data)
    response['MisconceptionId'] = miscon_id
    response['round_seed'] = round_seed
    
    return response

In [22]:
import itertools
inputs = []
for r in range(50):
    for i in df_mapping.MisconceptionId.tolist():
        inputs.append([i, r])

inputs = list(inputs)

#### TEST GEN 

In [23]:
output = generate(inputs[18000])
fprint(output)

PLANING,1. Identify the misconception where students confuse the plan view (top view) with the side view. 2. Create problems involving 3D shapes and their different views. 3. Formulate a problem where the correct solution involves identifying the top view of a 3D shape. 4. Design incorrect options reflecting the mistake of identifying the side view as the top view.
QUESTION,"Question: Which of the following is the plan view (top view) of the 3D shape shown below? ![A 3D shape composed of a rectangular prism with a smaller rectangular prism on top of it, aligned to one side. The dimensions are not labeled, but the shape is clearly visible.]() A. ![A side view of the 3D shape, showing the smaller rectangular prism on top of the larger one.] B. ![A top view of the 3D shape, showing the outline of the larger rectangular prism with the smaller one on top.] C. ![A front view of the 3D shape, showing the larger rectangular prism with the smaller one on top.] SubjectName: 3D Names and Properties of Shapes ConstructName: Understand and draw the plan, front, and side views of 3D shapes"
CORRECT_ANSWER,"B. ![A top view of the 3D shape, showing the outline of the larger rectangular prism with the smaller one on top.]"
STUDENT_WRONG_ANSWER,"A. ![A side view of the 3D shape, showing the smaller rectangular prism on top of the larger one.]"
MISCONCEPTION_NAME,Thinks the plan view is from the side of the shape
MisconceptionId,2478
round_seed,6


In [24]:
# lm.ih()

### RUN MULTI THREAD

In [25]:
len(inputs)

129350

In [27]:
generated_items = multi_thread(generate, inputs, 1)

Multi-thread, Function: `gener..:   0%|                              | 387/129350 [00:01<10:57, 196.03it/s, SUCCESS=405]

ERROR    | thread.py:72 (_handle_results) - First error of type ValueError: Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.11/site-packages/speedy_utils/multi_worker/thread.py", line 48, in _process_single_task
    ret = func(inp) if not isinstance(inp, dict) else func(**inp)
                                                      ^^^^^^^^^^^
  File "/tmp/ipykernel_38086/884461019.py", line 38, in generate
    response = generate_problem(input_data)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_38086/1971947819.py", line 39, in generate_problem
    pred = prog(MISCONCEPTION_NAME=input["miscon"], demos=input["demos"])
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.local/lib/python3.11/site-packages/dspy/utils/callback.py", line 202, in wrapper
    return fn(instance, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.local/lib/python3.11/site-packages/dspy/predict

Multi-thread, Function: `gener..:  84%|██▌| 108421/129350 [36:12<06:59, 49.90it/s, SUCCESS=108309, Error_ValueError=112]


Multi thread results:
+------------------+---------+
| Key              |   Value |
+==================+=========+
| SUCCESS          |  108309 |
+------------------+---------+
| Error_ValueError |     112 |
+------------------+---------+


In [28]:
items = [i for i in generated_items if isinstance(i, dict)]
len(items)

108309

In [38]:

df = pd.DataFrame(items)

In [39]:
# df = pd.read_parquet('./data/synthetic/df_generated_problems_72b_108k.parquet')

In [46]:
df['PROMPT'] = df.apply(lambda row:TEMPLATE_INPUT_V3.format(**row), 1)
df.to_parquet('./data/synthetic/df_generated_problems_72b_108k.parquet')

In [47]:
# for q in df[df['MisconceptionId']==0]['QUESTION'].tolist():
#     print(q)
#     print('====')

NOW RUN `python embed/synthetic_data_to_train_items.py`

In [49]:
!rm './data/embeddings/108k_synth_4k3_real.jsonl'

In [50]:
!python embed/synthetic_data_to_train_items.py './data/synthetic/df_generated_problems_72b_108k.parquet' './data/embeddings/raw_108k_synthetic_query_positive_raw.jsonl'

2024-12-04 15:26:26.855 | INFO     | eedi.common:<module>:429 - common: ['VectorDBRetriever', 'prepare_db_train_val', 'preproc_df', 'get_df_parsed', 'format_df_generic', 'compute_metric', 'mine_hard_negatives', 'mine_hard_negatives', 'extract_tag', 'format_input_v2', 'format_input_v3', 'TEMPLATE_INPUT_V3', 'construct_base_prompt_v3', 'get_synthetic_df', 'format_docs', 'get_parsed_df_v3', 'df_mapping', 'get_parsed_df_v4', 'get_openai_misconception_db', 'update_train_df', 'load_latest_df', 'OpenAIModel', 'LocalLM']
Saved 108309 items to ./data/embeddings/raw_108k_synthetic_query_positive_raw.jsonl


In [53]:
# pd.read_csv('./data/train_4k3_with_subjectName.csv')

In [55]:
!python embed/import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
.py './data/train_4k3_with_subjectName.csv' './data/embeddings/raw_4k3_real.jsonl'

2024-12-04 16:45:35.654 | INFO     | eedi.common:<module>:429 - common: ['VectorDBRetriever', 'prepare_db_train_val', 'preproc_df', 'get_df_parsed', 'format_df_generic', 'compute_metric', 'mine_hard_negatives', 'mine_hard_negatives', 'extract_tag', 'format_input_v2', 'format_input_v3', 'TEMPLATE_INPUT_V3', 'construct_base_prompt_v3', 'get_synthetic_df', 'format_docs', 'get_parsed_df_v3', 'df_mapping', 'get_parsed_df_v4', 'get_openai_misconception_db', 'update_train_df', 'load_latest_df', 'OpenAIModel', 'LocalLM']
Saved 4370 items to ./data/embeddings/raw_4k3_real.jsonl
